Many systems have a "critical point": where the behaviour fundamentally changes from one type to another. In this code we will simulate wildfires and observe this critical point: most wildfires are small and damage few trees, but one there are enough trees the fire becomes devastating and catastrophic. The criticality is also interesting here because it is self-organising: the tree growth nudges the system towards criticality until the devastation happens. 

See: https://www.youtube.com/watch?v=HBluLfX2F_k

In [1]:
import numpy as np
import random as r
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
import imageio.v2 as imageio 
r.seed(1)

The forest is a 250 x 250 grid of squares. It starts completely empty. If a square is empty, it has a probability p of growing a tree. Each time step each tree has a probability q of catching fire. If a tree catches fire, all adjacent trees catch fire at the next timestep. 

In [2]:
N = 250
p = 0.005
q = 0.00005

States:

-2: currently burning

-1: recently burnt 

0 : Empty

1 : Tree

In [3]:
forest = [
    [0 for _ in range(N)] for __ in range(N)
]

def growTrees(forest):
    output = [row[:] for row in forest]
    for i in range(len(forest)):
        for j in range(len(forest[i])):
            if forest[i][j] == 0:
                if r.uniform(0, 1) < p:
                    output[i][j] = 1
    return(output)


def igniteTrees(forest):
    output = [row[:] for row in forest]
    for i in range(len(forest)):
        for j in range(len(forest[i])):
            if forest[i][j] == 1:
                if r.uniform(0, 1) < q:
                    output[i][j] = -2
    return(output)


def spreadFire(forest):
    n = len(forest)
    output = [row[:] for row in forest]
    neighbours = [(1, 0), (-1, 0), (0, 1), (0, -1)]
    for i in range(n):
        for j in range(len(forest[i])):
            if forest[i][j] == -2:
                output[i][j] = -1
                for di, dj in neighbours:
                    ni, nj = i + di, j + dj
                    if 0 <= ni < n and 0 <= nj < len(forest[ni]):
                        if forest[ni][nj] == 1:
                            output[ni][nj] = -2
    return(output)

def burnOut(forest):
    output = [row[:] for row in forest]
    for i in range(len(forest)):
        for j in range(len(forest[i])):
            if forest[i][j] == -1:
                output[i][j] = 0
    return(output)

def updateForest(forest):
    output = growTrees(forest)
    output = igniteTrees(output)
    output = spreadFire(output)
    output = burnOut(output)
    return(output)

In [4]:
def showForest(forest, title=None):
    arr = np.array(forest)
    cmap = ListedColormap([
        'red',         # -2: on fire
        'black',       # -1: burnt out
        'saddlebrown', #  0: empty
        'green',       #  1: tree
    ])
    bounds = [-2.5, -1.5, -0.5, 0.5, 1.5]
    norm = BoundaryNorm(bounds, cmap.N)
    plt.figure(figsize=(5, 5))
    plt.imshow(arr, cmap=cmap, norm=norm, interpolation='nearest')
    plt.axis('off')
    if title is not None:
        plt.title(title)
    plt.tight_layout()
    plt.show()
    
cmap = ListedColormap([
    'red',         # -2: on fire
    'black',       # -1: burnt out
    'saddlebrown', #  0: empty
    'green',       #  1: tree
])
bounds = [-2.5, -1.5, -0.5, 0.5, 1.5]
norm = BoundaryNorm(bounds, cmap.N)


def forestFrame(forest, title=None):
    arr = np.array(forest)

    fig, ax = plt.subplots(figsize=(5, 5))
    ax.imshow(arr, cmap=cmap, norm=norm, interpolation='nearest')
    ax.axis('off')
    if title is not None:
        ax.set_title(title)

    fig.canvas.draw()
    image = np.asarray(fig.canvas.buffer_rgba())
    plt.close(fig)
    return image


def makeGif(forest, steps=200, frame_duration=0.05, path="forest.gif"):
    images = []
    current = [row[:] for row in forest]
    for t in tqdm(range(steps)):
        img = forestFrame(current, title=f"Step {t}")
        images.append(img)
        current = updateForest(current)
    imageio.mimsave(path, images, duration=frame_duration)
    print(f"Saved GIF to {path}")
makeGif(forest, steps=1000, frame_duration=0.03, path="forest.gif")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [01:13<00:00,  6.76it/s]


Saved GIF to forest.gif


Notice how there are frequently very small fires but occasionally (e.g. arround step 200 and step 430) we get very large fires which wipe everything out. The fewer fires there are the more trees there are, and the more trees there are the more likely a very large fire becomes. So the system becomes self-organising towards criticality: it builds up towards an unusually large fire, burns itself down, then rebuilds until it can burn down again.